In [ ]:
pip show azureml-contrib-fairness

In [ ]:
pip install --upgrade fairlearn==0.7.0 raiwidgets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

print("Loading Data...")
data = pd.read_csv('data/titanic.csv')

features = ['PassengerId','Pclass','Sex','Age','SibSp','Parch', 'Fare', 'Embarked']
X, y = data[features].values, data['Survived'].values

S = data[['Age']].astype(int)
S['Age'] = np.where(S.Age > 50, 'Over 50', '50 or younger')

X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.20, random_state=0, stratify=y)

print("Training model...")
titanic_model = DecisionTreeClassifier().fit(X_train, y_train)

print("Model trained.")

In [ ]:
from fairlearn.metrics import selection_rate, MetricFrame
from sklearn.metrics import accuracy_score, recall_score, precision_score

y_hat = titanic_model.predict(X_test)

print("Overall Metrics:")
overall_selection_rate = selection_rate(y_test, y_hat)
print("\tSelection Rate:", overall_selection_rate)
overall_accuracy = accuracy_score(y_test, y_hat)
print("\tAccuracy:", overall_accuracy)
overall_recall = recall_score(y_test, y_hat)
print("\tRecall:", overall_recall)
overall_precision = precision_score(y_test, y_hat)
print("\tPrecision:", overall_precision)

print('\nMetrics by Group:')
metrics = {'selection_rate': selection_rate,
           'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score}

group_metrics = MetricFrame(metrics=metrics,
                             y_true=y_test,
                             y_pred=y_hat,
                             sensitive_features=S_test['Age'])

print(group_metrics.by_group)

# Exercise: go through the dasboard to understand the disparities

In [ ]:
from raiwidgets import FairnessDashboard

FairnessDashboard(sensitive_features=S_test,
                   y_true=y_test,
                   y_pred={"titanic_model": titanic_model.predict(X_test)})